In [ ]:
import sys
import os
import torch
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# Add the parent directory to path so we can import from src
sys.path.append(os.path.abspath(os.path.join('..')))

from src.model import build_model
from src.sensor import predict_single_image, estimate_continuous_value
from src.visualization.activation import ActivationMapGenerator

# Configuration
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MODEL_TYPE = 'Lrgbd'    # Options: 'rgb', 'depth', 'Lrgbd'
TARGET_TYPE = 'angle'   # Options: 'angle', 'rad'
WEIGHTS_PATH = '../results/models/Lrgbd_angle_v3_best.pth' # Point to your trained weights
IMAGE_DIR = '../dataset/final_split/test' # Point to test images

print(f"Running on {DEVICE}")

In [ ]:
# Determine classes
num_classes = 16 if TARGET_TYPE == 'angle' else 6

# Build and Load
model = build_model(MODEL_TYPE, num_classes)
try:
    model.load_state_dict(torch.load(WEIGHTS_PATH, map_location=DEVICE))
    print("✅ Model loaded successfully!")
except FileNotFoundError:
    print("❌ Weights not found. Please check the path.")

model.to(DEVICE)
model.eval();

In [ ]:
# Choose a sample image
img_name = os.listdir(os.path.join(IMAGE_DIR, 'rgb'))[0] # Take first image
rgb_path = os.path.join(IMAGE_DIR, 'rgb', img_name)
depth_path = os.path.join(IMAGE_DIR, 'depth', img_name)

print(f"Testing on: {img_name}")

# Load and Preprocess
rgb_cv = cv2.imread(rgb_path)
rgb_cv = cv2.cvtColor(rgb_cv, cv2.COLOR_BGR2RGB)
depth_cv = cv2.imread(depth_path, cv2.IMREAD_UNCHANGED)

# Prepare Tensors
from torchvision import transforms
t_rgb = transforms.ToTensor()(rgb_cv).unsqueeze(0).to(DEVICE)
t_depth = torch.from_numpy(depth_cv.astype("float32") / 65535.0).unsqueeze(0).unsqueeze(0).to(DEVICE)

# Prepare input based on model type
if MODEL_TYPE == 'Lrgbd':
    input_tensor = [t_rgb, t_depth] # List for Late Fusion
else:
    input_tensor = t_rgb if MODEL_TYPE == 'rgb' else t_depth

# Show Image
plt.figure(figsize=(5,5))
plt.imshow(rgb_cv)
plt.title("Input Image")
plt.axis('off')
plt.show()

In [ ]:
# 1. Get Probability Vector
if MODEL_TYPE == 'Lrgbd':
    # Manual forward because our helper expects single tensor usually
    with torch.no_grad():
        out = model(t_rgb, t_depth)
        probs = torch.nn.functional.softmax(out, dim=1)[0].cpu().numpy()
else:
    probs = predict_single_image(model, input_tensor, DEVICE)

# 2. Interpolate Value
prediction = estimate_continuous_value(probs, TARGET_TYPE)

print(f"Predicted {TARGET_TYPE.capitalize()}: {prediction:.2f}")

# 3. Visualize Probabilities
plt.figure(figsize=(10, 3))
plt.bar(range(len(probs)), probs)
plt.title("Class Probabilities")
plt.xlabel("Class Index")
plt.ylabel("Softmax Score")
plt.show()

In [ ]:
# Initialize Generator
cam_gen = ActivationMapGenerator(model, MODEL_TYPE, DEVICE)

# Generate
heatmap = cam_gen.generate(input_tensor, input_img_for_overlay=Image.fromarray(rgb_cv))

# Show
plt.figure(figsize=(6,6))
plt.imshow(heatmap)
plt.title(f"Where is the network looking? ({MODEL_TYPE})")
plt.axis('off')
plt.show()